## Aircraft Control System


**Purpose:**


**What you will learn:**




In [1]:
from falco.core.vehicle.components.component import Component
from falco.core.vehicle.controls.vehicle_control_system import VehicleControlSystem, ControlSurface, PropulsiveControl, PropulsiveControlRPM
import csdl_alpha as csdl
import numpy as np
from typing import List

In [2]:
class C172Control(VehicleControlSystem):

    def __init__(self, elevator_component, aileron_right_component, aileron_left_component, rudder_component, symmetrical: bool = True):
        self.symmetrical = symmetrical

        self.elevator = ControlSurface('elevator_left', lb=-26, ub=28, component=elevator_component)
        if not symmetrical:
            self.aileron_left = ControlSurface('aileron_left', lb=-15, ub=20, component=aileron_right_component)
            self.aileron_right = ControlSurface('aileron_right', lb=-15, ub=20, component=aileron_left_component)
        else:
            self.aileron = ControlSurface('aileron', lb=-15, ub=20, component=aileron_right_component)
        self.rudder = ControlSurface('rudder', lb=-16, ub=16, component=rudder_component)

        self.engine = PropulsiveControl(name='engine', throttle=0.85)

        if symmetrical:
            self.u = csdl.concatenate((self.aileron.deflection,
                                      -self.aileron.deflection,
                                      self.elevator.deflection,
                                      self.rudder.deflection,
                                      self.engine.throttle), axis=0)
        else:
            self.u = csdl.concatenate((self.aileron_left.deflection,
                                      self.aileron_right.deflection,
                                      self.elevator.deflection,
                                      self.rudder.deflection,
                                      self.engine.throttle), axis=0)

        if symmetrical:
            super().__init__(pitch_control=[self.elevator],
                             roll_control=[self.aileron],
                             yaw_control=[self.rudder],
                             throttle_control=[self.engine])
        else:
            super().__init__(pitch_control=[self.elevator],
                             roll_control=[self.aileron_left, self.aileron_right],
                             yaw_control=[self.rudder],
                             throttle_control=[self.engine])

    @property
    def control_order(self) -> List[str]:
        return ['roll', 'pitch', 'yaw', 'throttle']

    @property
    def lower_bounds(self):
        lb_elevator = self.elevator.lower_bound
        lb_rudder = self.rudder.lower_bound
        lb_thr = self.engine.lower_bound

        if self.symmetrical:
            lb_aileron_left = self.aileron.lower_bound
            lb_aileron_right = self.aileron.lower_bound
            return np.array([lb_aileron_left, lb_aileron_right, lb_elevator, lb_rudder, lb_thr])
        else:
            lb_aileron_left = self.aileron_left.lower_bound
            lb_aileron_right = self.aileron_right.lower_bound
            return np.array([lb_aileron_left, lb_aileron_right, lb_elevator, lb_rudder, lb_thr])

    @property
    def upper_bounds(self):
        ub_elevator = self.elevator.upper_bound
        ub_rudder = self.rudder.upper_bound
        ub_thr = self.engine.upper_bound

        if self.symmetrical:
            ub_aileron_left = self.aileron.upper_bound
            ub_aileron_right = self.aileron.upper_bound
            return np.array([ub_aileron_left, ub_aileron_right, ub_elevator, ub_rudder, ub_thr])
        else:
            ub_aileron_left = self.aileron_left.upper_bound
            ub_aileron_right = self.aileron_right.upper_bound
            return np.array([ub_aileron_left, ub_aileron_right, ub_elevator, ub_rudder, ub_thr])

In [ ]:
recorder = csdl.Recorder(inline=True, expand_ops=True, debug=False)
recorder.start()

c172_controls = C172Control(
    elevator_component=None,
    aileron_right_component=None,
    aileron_left_component=None,
    rudder_component=None,
    symmetrical=True
) # These are all None because we did not define any control surfaces in the previous ex_geometry.ipynb file.